# 3.3 Other interpretations of the transportation model

In [ ]:
# install dependencies
%pip install -q amplpy pandas

from amplpy import AMPL, ampl_notebook
import pandas as pd

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics

As the name suggests, a transportation model is applicable
whenever some material is being shipped from a set of origins to a set of
destinations.  Given certain amounts available at the origins, and required
at the destinations, the problem is to meet the requirements at a
minimum shipping cost.

Viewed more broadly, transportation models do not have to be
concerned with the shipping of "materials".  They can be applied to the
transportation of anything, provided that the quantities available and
required can be measured in some units, and that the transportation cost
per unit can be determined.  They might be used to model the shipments of
automobiles to dealers, for example, or the movement of military personnel
to new assignments.

In an even broader view, transportation models need not
deal with "shipping" at all.  The quantities at the origins may
be merely associated with various destinations, while the objective
measures some value of the association that has nothing to do
with actually moving anything.  Often the result is referred to as an
"assignment" model.

As one particularly well-known example,
consider a department that needs to assign some number of
people to an equal number of offices.  The origins now represent
individual people, and the destinations represent individual offices.
Since each person is assigned one office, and each office is occupied
by one person, all of the parameter values
`supply[i]`
and
`demand[j]`
are 1.
We interpret
`Trans[i,j]`
as the "amount" of person
`i`
that is
assigned to office
`j`  ;
that is, if
`Trans[i,j]`
is 1 then person
`i`
will
occupy office
`j` ,
while if
`Trans[i,j]`
is 0 then person
`i`
will not occupy
office
`j` .

What of the objective?  One possibility is to ask people to
rank the offices, giving their first choice, second choice, and so
forth.  Then we can let
`cost[i,j]`
be the rank that person
`i`
gives to office
`j` .
This convention lets each objective function term
`cost[i,j]
*
Trans[i,j]`
represent the preference of person
`i`
for office
`j` ,
if person
`i`
is assigned to office
`j`
(`Trans[i,j]`
equals 1),
or zero if person
`i`
is not assigned to office
`j`
(`Trans[i,j]` 
equals 0).
Since the
objective is the sum of all these terms,
it must equal the sum of all the nonzero terms, which is the sum of
everyone's rankings for the offices to which they were assigned.  By
minimizing this sum, we can hope to find an assignment that will please
a lot of people.

<a id='fig-3-2'><center><b>Figure 3-2:</b>  Data for assignment problem.</center></a>

In [2]:
ORIG = ['Coullard', 'Daskin', 'Hazen', 'Hopp', 'Iravani', 'Linetsky', 'Mehrotra', 'Nelson', 'Smilowitz', 'Tamhane', 'White']

DEST = ['C118', 'C138', 'C140', 'C246', 'C250', 'C251', 'D237', 'D239', 'D241', 'M233', 'M239']

# List of "ORIG" number of elements, all of them being 1
supply = [1] * len(ORIG)

# List of "DEST" number of elements, all of them being 1
demand = [1] * len(DEST)

df_cost = pd.DataFrame(
    [
        ['Coullard', 6, 9, 8, 7, 11, 10, 4, 5, 3, 2, 1],
        ['Daskin', 11, 8, 7, 6, 9, 10, 1, 5, 4, 2, 3],
        ['Hazen', 9, 10, 11, 1, 5, 6, 2, 7, 8, 3, 4],
        ['Hopp', 11, 9, 8, 10, 6, 5, 1, 7, 4, 2, 3],
        ['Iravani', 3, 2, 8, 9, 10, 11, 1, 5, 4, 6, 7],
        ['Linetsky', 11, 9, 10, 5, 3, 4, 6, 7, 8, 1, 2],
        ['Mehrotra', 6, 11, 10, 9, 8, 7, 1, 2, 5, 4, 3],
        ['Nelson', 11, 5, 4, 6, 7, 8, 1, 9, 10, 2, 3],
        ['Smilowitz', 11, 9, 10, 8, 6, 5, 7, 3, 4, 1, 2],
        ['Tamhane', 5, 6, 9, 8, 4, 3, 7, 10, 11, 2, 1],
        ['White', 11, 9, 8, 4, 6, 5, 3, 10, 7, 2, 1]
    ],
    columns=['ORIG', 'C118', 'C138', 'C140', 'C246', 'C250', 'C251', 'D237', 'D239', 'D241', 'M233', 'M239']
).set_index('ORIG')

To use the transportation model for this purpose, we need only
supply the appropriate data.
[Figure 3-2](#fig-3-2) is one example, with 11 people to be
assigned to 11 offices.

<!--
The
`default`
option has been used to set all the
`supply`
and
`demand`
values to 1 without typing all the 1's.  If we store this data set in
`assign.dat` ,
we can use it with the transportation model that we already
have:
-->

In [3]:
ampl = AMPL()
ampl.read('transp.mod')
ampl.set['ORIG'] = ORIG
ampl.set['DEST'] = DEST
ampl.param['supply'] = supply
ampl.param['demand'] = demand
ampl.param['cost'] = df_cost
ampl.option['solver'] = 'highs'
ampl.solve()

HiGHS 1.6.0: HiGHS 1.6.0: optimal solution; objective 28
19 simplex iterations
0 barrier iterations
 


By setting the
option
`omit_zero_rows`
to 1,
we can print just the nonzero terms in the objective.
(Options for displaying results are presented in Chapter @Display@ xTODO.)
This
listing tells us each person's assigned room and his or her
preference for it:

In [4]:
ampl.option['omit_zero_rows'] = 1
ampl.display('Trans')

Trans :=
Coullard  D241   1
Daskin    D237   1
Hazen     C246   1
Hopp      C251   1
Iravani   C138   1
Linetsky  C250   1
Mehrotra  D239   1
Nelson    C140   1
Smilowitz M233   1
Tamhane   C118   1
White     M239   1
;



The solution is reasonably successful, although it does assign two
fourth choices and one sixth choice.

It is not hard to see that when all the
`supply[i]`
and
`demand[j]`
values are 1, any
`Trans[i,j]`
satisfying all the constraints
must be between 0 and 1.  But how did we know that every
`Trans[i,j]`
would equal either 0 or 1 in the optimal solution, rather than, say,
\(12?  We were able to rely on a special property of transportation
models, which guarantees that as long as all supply and demand values
are integers, and all lower and upper bounds on the variables are integers,
there will be an optimal solution that is entirely integral.
Moreover, we used a solver that always finds one of these integral
solutions.  But don't let this favorable result mislead you into assuming
that integrality can be assured in all other circumstances; even in
examples that seem to be much like the transportation model, finding
integral solutions can require a special solver, and a lot more work.
Chapter @Integer@ xTODO discusses issues of integrality at length.

A problem of assigning 100 people to 100 rooms has ten thousand
variables; assigning 1000 people to 1000 rooms yields a million variables.
In applications on this scale, however, most of the assignments can be
ruled out in advance, so that the number of actual decision variables is not
too large.  After looking at an initial solution, you may want to
rule out some more assignments 
&ndash;
in our example, perhaps no
assignment to lower than fifth choice should be allowed 
&ndash;
or you may
want to force some assignments to be made a certain way, in order to see
how the rest could be done optimally.  These situations require models
that can deal with subsets of pairs (of people and offices, or origins and
destinations) in a direct way.  `AMPL`'s features for describing
pairs and other "compound" objects are the subject of Chapter
@Sets2@. xTODO